In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn as sns
import helpers.functions as fn
import toolz
train_url = f'https://raw.githubusercontent.com/Mschaid/kg_workflow/main/test.csv'
test_url = f'https://raw.githubusercontent.com/Mschaid/kg_workflow/main/test.csv'

In [13]:
train, test = pd.read_csv(train_url), pd.read_csv(test_url)

In [14]:
train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [35]:
def process_data(df):
    dum_cols = ['Age_categories', 'Fare_categories', 'Title', 'Cabin_type', 'Sex']
    return(df
           .pipe(fn.process_missing)
           .pipe(fn.process_age)
           .pipe(fn.process_fare)
           .pipe(fn.process_cabin)
           .pipe(fn.process_titles)
           .pipe(fn.create_dummies, col)
           )
 
 

In [36]:
process_data(train)

TypeError: create_dummies() missing 1 required positional argument: 'column_name'